In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
reviews_df= pd.read_csv('amazon_alexa.tsv', sep = '\t')

In [ ]:
print(reviews_df)

In [ ]:
reviews_df.info()

In [ ]:
reviews_df.describe()

In [ ]:
reviews_df['verified_reviews']

In [ ]:
sns.heatmap(reviews_df.isnull(), yticklabels = False, cbar = False, cmap = 'Blues')

In [ ]:
reviews_df.hist(bins = 30 , figsize = (13,5), color = 'r')

In [ ]:
reviews_df['length'] = reviews_df['verified_reviews'].apply(len)

In [ ]:
reviews_df.head()

In [ ]:
reviews_df['length'].plot(bins = 100, kind = 'hist')

In [ ]:
reviews_df.length.describe()

In [ ]:
#longest
reviews_df[reviews_df['length'] == 2851]['verified_reviews'].iloc[0]

In [ ]:
#shortest
reviews_df[reviews_df['length'] == 1]['verified_reviews'].iloc[0]

In [ ]:
#average/mean
reviews_df[reviews_df['length'] == 132]['verified_reviews'].iloc[0]

In [ ]:
#postive reviews
postive = reviews_df[reviews_df['feedback']==1]

In [ ]:
#negative reviews
negative = reviews_df[reviews_df['feedback']==0]

In [ ]:
sns.countplot(reviews_df['feedback'], label = 'count')

In [ ]:
sns.countplot(x = 'rating' , data = reviews_df)

In [ ]:
plt.figure(figsize=(40,15))
sns.barplot(x = 'variation' , y = 'rating' , data = reviews_df, palette = 'deep')

In [ ]:
sentences = reviews_df['verified_reviews'].tolist()
sentences

In [ ]:
sentences_as_one_string = " ".join(sentences)

In [ ]:
sentences_as_one_string

In [ ]:
from wordcloud import WordCloud
plt.figure(figsize = (20,20))
plt.imshow(WordCloud().generate(sentences_as_one_string))

In [ ]:
negative_list = negative['verified_reviews'].tolist()
negative_list

In [ ]:
negative_sentences_as_one_string = " ".join(negative_list)
negative_sentences_as_one_string 

In [ ]:
plt.figure (figsize = (20,20))
plt.imshow(WordCloud().generate(negative_sentences_as_one_string))

In [ ]:
reviews_df.head()

In [ ]:
reviews_df = reviews_df.drop (['date', 'rating','length'],axis = 1)

In [ ]:
variation_dummies = pd.get_dummies(reviews_df['variation'], drop_first = True)

In [ ]:
print (variation_dummies)

In [ ]:
reviews_df.drop(['variation'], axis =1 , inplace = True)

In [ ]:
reviews_df = pd.concat([reviews_df,variation_dummies], axis =1)

In [ ]:
print (reviews_df)

In [ ]:
import string
string.punctuation

In [ ]:
#removing punctuation marks
reviews_df['punctuaution_removed']=reviews_df['verified_reviews'].str.replace('[^\w\s]', '')

In [ ]:
reviews_df.head()

In [ ]:
reviews_df.drop(['verified_reviews'], axis =1 , inplace = True)

In [ ]:
reviews_df.head()

In [ ]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words

In [ ]:
reviews_df['punctuaution_removed_cleaned']=reviews_df['punctuaution_removed'].apply(lambda x: " ".join(word for word in x.split() if word not in stop_words) )

In [ ]:
reviews_df.head()

In [ ]:
#value counts of words top 30
pd.Series(" ".join(reviews_df['punctuaution_removed_cleaned']).split()).value_counts()[:60]

In [ ]:
other_stop_words=['I','Im', 'We','It','test','this','So','even','far', 'Its','this','also']
len(other_stop_words)

In [ ]:
reviews_df['cleanview']=reviews_df['punctuaution_removed_cleaned'].apply(lambda x: " ".join(word for word in x.split() if word not in other_stop_words))

In [ ]:
reviews_df.head()

In [ ]:
from textblob import Word

In [ ]:
reviews_df['lemmatized']=reviews_df['cleanview'].apply(lambda x: " ".join( Word(word).lemmatize() for word in x.split()))

In [ ]:
reviews_df.head()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
vectorizer = CountVectorizer()

In [ ]:
reviews_count_vectorizer = vectorizer.fit_transform(reviews_df['lemmatized'])

In [ ]:
# print (vectorizer.get_feature_names())

In [ ]:
print (reviews_count_vectorizer.toarray())

In [ ]:
reviews_count_vectorizer.shape

In [ ]:
reviews_df.drop(['punctuaution_removed','punctuaution_removed_cleaned','cleanview','lemmatized'], axis = 1, inplace = True)

In [ ]:
reviews = pd.DataFrame(reviews_count_vectorizer.toarray())

In [ ]:
reviews_df = pd.concat([reviews_df,reviews], axis = 1)

In [ ]:
reviews_df.head()

In [ ]:
X = reviews_df.drop(['feedback'], axis = 1)

In [ ]:
y = reviews_df['feedback']

In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X, y, test_size = 0.2)

In [ ]:
from sklearn.naive_bayes import MultinomialNB

In [ ]:
NB_classifier = MultinomialNB()
model = NB_classifier.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
y_predict_train = model.predict(X_train)

In [ ]:
y_predict_train

In [ ]:
cm = confusion_matrix(y_train, y_predict_train)
sns.heatmap(cm, annot = True)

In [ ]:
y_predict_test = model.predict(X_test)
cm = confusion_matrix(y_test, y_predict_test)
sns.heatmap(cm, annot = True)


In [ ]:
print (classification_report(y_test, y_predict_test))

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


In [ ]:
model = LogisticRegression()
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
print ('Acuracy = {}%'.format(100*accuracy_score(y_pred,y_test)))

In [ ]:
cm = confusion_matrix(y_pred,y_test)
sns.heatmap(cm,annot = True)

In [ ]:
print (classification_report (y_test, y_pred))